In [1]:
%config IPCompleter.greedy=True

In [2]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, Module

ws = Workspace.get(name='jietong-test-westeurope', subscription_id='f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2',
                   resource_group='studio-v2-test')

In [6]:
# Load modules from workspace
train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Train')
score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')

# Load a module from local as anonymous module
#a_module_in_development_func = Module.load_from_yaml(ws, path='/home/code/new_module/spec.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [18]:
help(train_module_func)

Help on function microsoft.com/aml/samples://Train in microsoft:

microsoft.com/aml/samples://Train(training_data: 'Training data organized in the torchvision format/structure' = None, max_epochs: 'Maximum number of epochs for the training, default is 5' = '5', learning_rate: 'Learning rate, default is 0.01' = '0.01')
    :param training_data: Training data organized in the torchvision format/structure
    :param max_epochs: Maximum number of epochs for the training, default is 5
    :param learning_rate: Learning rate, default is 0.01



In [16]:
train = train_module_func(
    training_data=train_data, 
    max_epochs=5, 
    learning_rate=0.02)

score = score_module_func(
    model_input=train.outputs.model_output, 
    test_data=test_data)

eval = eval_module_func(scoring_result=score.outputs.score_output)

pipeline = Pipeline(nodes=[train, score, eval], outputs=eval.outputs)

run = pipeline.submit_run(
    ws, 
    experiment_name='sample-pipelines', 
    default_compute_target='cpu-cluster')

run.wait_for_completion()

Submitted PipelineRun 591b6ae5-34c9-4767-8a42-66264734f151
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sample-pipelines/runs/591b6ae5-34c9-4767-8a42-66264734f151?wsid=/subscriptions/f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2/resourcegroups/studio-v2-test/workspaces/jietong-test-westeurope
PipelineRunId: 591b6ae5-34c9-4767-8a42-66264734f151
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sample-pipelines/runs/591b6ae5-34c9-4767-8a42-66264734f151?wsid=/subscriptions/f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2/resourcegroups/studio-v2-test/workspaces/jietong-test-westeurope
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 85d6f4ac-7f52-4180-87ad-869e7fa74497
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sample-pipelines/runs/85d6f4ac-7f52-4180-87ad-869e7fa74497?wsid=/subscriptions/f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2/resourcegroups/studio-v2-test/workspaces/jietong-test-westeurope
StepRun( Train ) Stat

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}